In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy import stats
import sys

In [2]:
train_data=pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/house-prices-advanced-regression-techniques/train.csv")
test=pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/house-prices-advanced-regression-techniques/test.csv")
Sale_Price=train_data.iloc[:,80]
Sale_Price.shape

(1460,)

In [3]:
train=train_data.drop(["SalePrice"],axis=1)

In [4]:
data= pd.concat([train,test], keys=['x', 'y'])#here X is training data and Y testing data
# data=data.drop(["Id"],axis=1)
data.shape

(2919, 80)

In [5]:
data =data.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

In [6]:
categorical_features = data.select_dtypes(include = ["object"]).columns
train_cat = data[categorical_features]


In [7]:
#delete highly skew category data
pct=[]
for ix in train_cat.columns:
    temp=train_cat[ix].describe()
    pct.append(temp["freq"]/temp["count"])
skewData=pd.DataFrame(pct,index=train_cat.columns,columns=["skewness"])
skewData=skewData.sort_values(by="skewness",ascending=False)
# print (skewData)
data = data.drop((skewData[skewData['skewness'] >= 0.95]).index,1) 
data.shape

(2919, 73)

In [8]:
train_data=data

In [9]:
#2.2: some categorical features when there is information in the order
#Alley: Type of alley access to property
train_data=train_data.replace({"Alley":{"Grvl" : 1, "Pave" : 2}})
#BsmtCond: Evaluates the general condition of the basement
train_data=train_data.replace({"BsmtCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#BsmtExposure: Refers to walkout or garden level walls
train_data=train_data.replace({"BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3}})
#BsmtFinType1: Rating of basement finished area
train_data=train_data.replace({"BsmtFinType1" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6}})
#BsmtFinType2: Rating of basement finished area (if multiple types)
train_data=train_data.replace({"BsmtFinType2" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6}})
#BsmtQual: Evaluates the height of the basement
train_data=train_data.replace({"BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})
#ExterCond: Evaluates the present condition of the material on the exterior
train_data=train_data.replace({"ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5}})
#ExterQual: Evaluates the quality of the material on the exterior 
train_data=train_data.replace({"ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5}})
#FireplaceQu: Fireplace quality
train_data=train_data.replace({"FireplaceQu" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#Functional: Home functionality (Assume typical unless deductions are warranted)
train_data=train_data.replace({"Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8}})
#HeatingQC: Heating quality and condition
train_data=train_data.replace({"HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#KitchenQual: Kitchen quality
train_data=train_data.replace({"KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#LandSlope: Slope of property
train_data=train_data.replace({"LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3}})
#LotShape: General shape of property
train_data=train_data.replace({"LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4}})
#PavedDrive: Paved driveway
train_data=train_data.replace({"PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2}})

#2.3: Simplifications of existing categorical 
train_data["Condition1"] = train_data.Condition1.replace({"RRNe" : "Other", 
                                                  "RRNn" : "Other","PosA" : "Other", 
                                                   "RRAe" : "Other"
                                                  })
train_data["Electrical"] = train_data.Electrical.replace({"Mix" : "Other", 
                                                  "FuseP" : "Other"
                                                  })
train_data["Exterior1st"] = train_data.Exterior1st.replace({"AsphShn" : "Other", 
                                                  "CBlock" : "Other","ImStucc" : "Other", 
                                                   "BrkComm" : "Other","Stone" : "Other"
                                                  })
train_data["Exterior2nd"] = train_data.Exterior2nd.replace({"CBlock" : "Other", 
                                                  "AsphShn" : "Other","Stone" : "Other", 
                                                   "Brk Cmn" : "Other","ImStucc" : "Other"
                                                  })
train_data["Foundation"] = train_data.Foundation.replace({"Wood" : "Other", 
                                                  "Stone" : "Other"
                                                  })
# train_data["GarageType"] = train_data.GarageType.replace({"2Types" : "Other", 
#                                                   "CarPort" : "Other"
#                                                   })
train_data["HouseStyle"] = train_data.HouseStyle.replace({"2.5Fin" : "Other", 
                                                  "2.5Unf" : "Other",
                                                  "1.5Unf" : "Other"
                                                  })
train_data["LotConfig"] = train_data.LotConfig.replace({"FR3" : "FR2"
                                                  })
train_data["MSSubClass"] = train_data.MSSubClass.replace({"SC40" : "Other", 
                                                  "SC180" : "Other",
                                                  "SC45" : "Other",
                                                  "SC75" : "Other"
                                                  })
train_data["MSZoning"] = train_data.MSZoning.replace({"C (all)" : "Other", 
                                                  "RH" : "Other"
                                                  })
train_data["Neighborhood"] = train_data.Neighborhood.replace({"Blueste" : "Other", 
                                                  "NPkVill" : "Other",
                                                  "Veenker" : "Other"
                                                  })
train_data["RoofStyle"] = train_data.RoofStyle.replace({"Shed" : "Other", 
                                                  "Mansard" : "Other",
                                                  "Gambrel" : "Other",
                                                  "Flat" : "Other"
                                                  })
train_data["SaleCondition"] = train_data.SaleCondition.replace({"AdjLand" : "Other", 
                                                  "Alloca" : "Other"
                                                  })
train_data["SaleType"] = train_data.SaleType.replace({"Con" : "Other", 
                                                  "Oth" : "Other",
                                                  "CWD" : "Other", "ConLI" : "Other",
                                                  "ConLw" : "Other","ConLD" : "Other"
                                                  })

#2.4 Combinations of existing features
# Overall quality of the house
train_data["OverallGrade"] = train_data["OverallQual"] * train_data["OverallCond"]
# Overall quality of the exterior
train_data["ExterGrade"] = train_data["ExterQual"] * train_data["ExterCond"]
# Overall kitchen score
train_data["KitchenScore"] = train_data["KitchenAbvGr"] * train_data["KitchenQual"]
# Total number of bathrooms
train_data["TotalBath"] = train_data["BsmtFullBath"] + (0.5 * train_data["BsmtHalfBath"]) + \
train_data["FullBath"] + (0.5 * train_data["HalfBath"])
train_data.drop(["BsmtFullBath","BsmtHalfBath","FullBath","HalfBath"],axis=1)
# Total SF for house (incl. basement)
train_data["AllSF"] = train_data["GrLivArea"] + train_data["TotalBsmtSF"]
train_data.drop(["GrLivArea","TotalBsmtSF"],axis=1)
# Total SF for 1st + 2nd floors
train_data["AllFlrsSF"] = train_data["1stFlrSF"] + train_data["2ndFlrSF"]
train_data.drop(["1stFlrSF","2ndFlrSF"],axis=1)
# Total SF for porch
train_data["AllPorchSF"] = train_data["OpenPorchSF"] + train_data["EnclosedPorch"] + \
train_data["3SsnPorch"] + train_data["ScreenPorch"]
train_data.drop(["OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"],axis=1)
# House completed before sale or not
train_data["BoughtOffPlan"] = train_data.SaleCondition.replace({"Abnorml" : 0, "Alloca" : 0, "AdjLand" : 0, 
                                                      "Family" : 0, "Normal" : 0, "Partial" : 1})

In [10]:
# data=train_data

In [11]:
# train_data["OverallQual-s2"] = train_data["OverallQual"] ** 2
# train_data["AllSF-2"] = train_data["AllSF"] ** 2
# train_data["AllFlrsSF-2"] = train_data["AllFlrsSF"] ** 2
# train_data["GrLivArea-2"] = train_data["GrLivArea"] ** 2
# # train_data["SimplOverallQual-s2"] = train_data["SimplOverallQual"] ** 2
# train_data["ExterQual-2"] = train_data["ExterQual"] ** 2
# train_data["GarageCars-2"] = train_data["GarageCars"] ** 2
# train_data["TotalBath-2"] = train_data["TotalBath"] ** 2
# train_data["KitchenQual-2"] = train_data["KitchenQual"] ** 2

In [12]:
train_data.head()

Id MSSubClass MSZoning  LotFrontage  LotArea  Alley  LotShape  \
x 0   1       SC60       RL         65.0     8450    NaN         4   
  1   2       SC20       RL         80.0     9600    NaN         4   
  2   3       SC60       RL         68.0    11250    NaN         3   
  3   4       SC70       RL         60.0     9550    NaN         3   
  4   5       SC60       RL         84.0    14260    NaN         3   

    LandContour LotConfig Neighborhood  ... SaleType SaleCondition  \
x 0         Lvl    Inside      CollgCr  ...       WD        Normal   
  1         Lvl       FR2        Other  ...       WD        Normal   
  2         Lvl    Inside      CollgCr  ...       WD        Normal   
  3         Lvl    Corner      Crawfor  ...       WD       Abnorml   
  4         Lvl       FR2      NoRidge  ...       WD        Normal   

    OverallGrade  ExterGrade  KitchenScore  TotalBath   AllSF AllFlrsSF  \
x 0           35          12           4.0        3.5  2566.0      1710   
  1           48           9           3.0        2.5  2524.0      1262   
  2           35          12           4.0        3.5  2706.0      1786   
  3           35           9           4.0        2.0  2473.0      1717   
  4           40          12           4.0        3.5  3343.0      2198   

    AllPorchSF BoughtOffPlan  
x 0         61             0  
  1          0             0  
  2         42             0  
  3        307             0  
  4         84             0  

[5 rows x 81 columns]

In [13]:
# Differentiate numerical features and categorical features
categorical_features = train_data.select_dtypes(include = ["object"]).columns
numerical_features = train_data.select_dtypes(exclude = ["object"]).columns
print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
train_num = train_data[numerical_features]
train_cat = train_data[categorical_features]
print("NAs for numerical features in train : " + str(train_num.isnull().values.sum()))
train_num = train_num.fillna(train_num.median())
print("Remaining NAs for numerical features in train : " + str(train_num.isnull().values.sum()))

Numerical features : 56
Categorical features : 25
NAs for numerical features in train : 5230
Remaining NAs for numerical features in train : 0


In [14]:
#2.6 turn category features to dummy
print("NAs for categorical features in train : " + str(train_cat.isnull().values.sum()))
train_cat = pd.get_dummies(train_cat, drop_first=True)
print("Remaining NAs for categorical features in train : " + str(train_cat.isnull().values.sum()))


NAs for categorical features in train : 8578
Remaining NAs for categorical features in train : 0


In [15]:
#2.7 join categorical and numerical features 
train_Data_New = pd.concat([train_num, train_cat], axis = 1)
print("New number of features : " + str(train_Data_New.shape[1]))

New number of features : 185


In [16]:
#2.8 remove collinear columns
# Create correlation matrix
corr_matrix = train_Data_New.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
# Drop features 
train_Data_New.drop(to_drop, axis=1, inplace=True)

In [17]:
#3.split in-sample data to training and validation sets
train_Data_New.index=train_Data_New["Id"]
train_Data_New.drop("Id", axis=1, inplace=True)
train_data.index=train_data["Id"]

In [18]:
train_Data_New.shape

(2919, 174)

In [19]:
train_Data_New.head()

,LotFrontage,LotArea,Alley,LotShape,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,...,MoSold_May,MoSold_Nov,MoSold_Oct,MoSold_Sep,SaleType_New,SaleType_Other,SaleType_WD,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Other
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,1.0,4,7,5,2003,2003,196.0,4,...,0,0,0,0,0,0,1,0,1,0
2,80.0,9600,1.0,4,6,8,1976,1976,0.0,3,...,1,0,0,0,0,0,1,0,1,0
3,68.0,11250,1.0,3,7,5,2001,2002,162.0,4,...,0,0,0,1,0,0,1,0,1,0
4,60.0,9550,1.0,3,7,5,1915,1970,0.0,3,...,0,0,0,0,0,0,1,0,0,0
5,84.0,14260,1.0,3,8,5,2000,2000,350.0,4,...,0,0,0,0,0,0,1,0,1,0


In [20]:
Train_data=train_Data_New[:1460]
Test_data=train_Data_New[1460:]

In [21]:
Train_data.shape

(1460, 174)

In [22]:
Train_data.insert(2,column="SalePrice",value=Sale_Price)

In [23]:
Train_data.shape

(1460, 175)

In [24]:
Test_data.shape

(1459, 174)

In [25]:
dependentV=Train_data["SalePrice"]
Train_data.drop("SalePrice", axis=1, inplace=True)

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [26]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression, SelectKBest
import numpy as np
from Help_functions_v2 import sklearn_Pvalue, sklearn_adjR2, RMSE

In [27]:
Train_data.head()

,LotFrontage,LotArea,Alley,LotShape,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,...,MoSold_May,MoSold_Nov,MoSold_Oct,MoSold_Sep,SaleType_New,SaleType_Other,SaleType_WD,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Other
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,1.0,4,7,5,2003,2003,196.0,4,...,0,0,0,0,0,0,1,0,1,0
2,80.0,9600,1.0,4,6,8,1976,1976,0.0,3,...,1,0,0,0,0,0,1,0,1,0
3,68.0,11250,1.0,3,7,5,2001,2002,162.0,4,...,0,0,0,1,0,0,1,0,1,0
4,60.0,9550,1.0,3,7,5,1915,1970,0.0,3,...,0,0,0,0,0,0,1,0,0,0
5,84.0,14260,1.0,3,8,5,2000,2000,350.0,4,...,0,0,0,0,0,0,1,0,1,0


In [28]:
#检查是否包含无穷数据
#False:包含
#True:不包含
print(np.isfinite(Train_data).all())
#False:不包含
#True:包含
print(np.isinf(Train_data).all())
 
#数据处理
train_inf = np.isinf(Train_data)
Train_data[train_inf] = 0

LotFrontage             True
LotArea                 True
Alley                   True
LotShape                True
OverallQual             True
                        ... 
SaleType_Other          True
SaleType_WD             True
SaleCondition_Family    True
SaleCondition_Normal    True
SaleCondition_Other     True
Length: 174, dtype: bool
LotFrontage             False
LotArea                 False
Alley                   False
LotShape                False
OverallQual             False
                        ...  
SaleType_Other          False
SaleType_WD             False
SaleCondition_Family    False
SaleCondition_Normal    False
SaleCondition_Other     False
Length: 174, dtype: bool
<ipython-input-28-f92436254c9c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [29]:
# 检查数据中是否有缺失值，以下两种方式均可
#Flase:对应特征的特征值中无缺失值
#True：有缺失值
print(Train_data.isnull().any())
print(np.isnan(Train_data).any())


LotFrontage             False
LotArea                 False
Alley                   False
LotShape                False
OverallQual             False
                        ...  
SaleType_Other          False
SaleType_WD             False
SaleCondition_Family    False
SaleCondition_Normal    False
SaleCondition_Other     False
Length: 174, dtype: bool
LotFrontage             False
LotArea                 False
Alley                   False
LotShape                False
OverallQual             False
                        ...  
SaleType_Other          False
SaleType_WD             False
SaleCondition_Family    False
SaleCondition_Normal    False
SaleCondition_Other     False
Length: 174, dtype: bool


In [30]:
X_train, X_test, y_train, y_test = train_test_split(Train_data, dependentV, test_size = 0.3, random_state = 0)
print("X_train : " + str(X_train.shape))
print("X_test : " + str(X_test.shape))
print("y_train : " + str(y_train.shape))
print("y_test : " + str(y_test.shape))

X_train : (1022, 174)
X_test : (438, 174)
y_train : (1022,)
y_test : (438,)


In [31]:
# how many % of data are missing
total = Train_data.isnull().sum().sort_values(ascending=False)
percent = (Train_data.isnull().sum()/Train_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
SaleCondition_Other,0,0.0
MSSubClass_SC120,0,0.0
MSSubClass_SC70,0,0.0
MSSubClass_SC60,0,0.0
MSSubClass_SC50,0,0.0
MSSubClass_SC30,0,0.0
MSSubClass_SC20,0,0.0
MSSubClass_SC190,0,0.0
MSSubClass_SC160,0,0.0
MSSubClass_SC150,0,0.0


In [32]:
#检查是否包含无穷数据
#False:包含
#True:不包含
print(np.isfinite(Train_data).all())
#False:不包含
#True:包含
print(np.isinf(Train_data).all())
#数据处理
train_inf = np.isinf(Train_data)
Train_data[train_inf] = 0

LotFrontage             True
LotArea                 True
Alley                   True
LotShape                True
OverallQual             True
                        ... 
SaleType_Other          True
SaleType_WD             True
SaleCondition_Family    True
SaleCondition_Normal    True
SaleCondition_Other     True
Length: 174, dtype: bool
LotFrontage             False
LotArea                 False
Alley                   False
LotShape                False
OverallQual             False
                        ...  
SaleType_Other          False
SaleType_WD             False
SaleCondition_Family    False
SaleCondition_Normal    False
SaleCondition_Other     False
Length: 174, dtype: bool
<ipython-input-32-f92436254c9c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [33]:
#3 feature selection with regularization
#3.1 Ridge Regularization
ridge_model=Ridge(alpha=1)
ridge_model.fit(X_train, y_train)
y_trainPred=ridge_model.predict(X_train)
#in sample
print("in-sample r-squared is")
print(r2_score(y_train, y_trainPred))
print ("RMSE is of in-sample")
print(RMSE(y_trainPred,y_train))
#out of sample
y_testPred=ridge_model.predict(X_test)
print("out-of-sample r-squared is")
print(r2_score(y_test, y_testPred))
print ("RMSE is out-of-sample")
print(RMSE(y_testPred,y_test))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').